# Smart Adaptive Triggering Mechanism (SATM) for Vibration-based SHM

## SATM Mechanism Overview
![SATM Overview](system.png)

## Project Structure
- System / Progress
- Controller
- Estimator
- Planning

## System Design

### System/Progress - *The System or Progress We Want to Control to Achieve Automation*
#### System/Progress - Analyses - Characteristics and Concerns
- Target: the triggering mechanism on accelerometer
- Goal: to fine-tune the parameters to help the mechanism to reach its highest F-beta Score
- Structure Simulation: to facilitate analyses, the structure is modeled by the state-space model, and we use the structure responses s.t. various excitation as the input to our triggering mechanism.
  - Structure Modelling
    - n DOF model
  - Excitation Simulation
    - Ambient Vibration
      - Gaussian Noise
    - Earthquake
      - Kanai-Tajimi
    - Impact
      - Spike-like Input
    - Strong Wind
      - Auto Regression
  - Response
  - Measurement Noise
    - SNR
- Key Issues:
  - No sufficient data in practice
  - The data is unbalanced
    - Uninterested Events > > Interested Events
  - Iteration interval determination
    - maybe the slope of the F-beta curve can be utilized
#### System/Progress - Solution
- Triggering Mechanism:
  - Window-A for Triggering
    - Height-A
    - Width-A
  - Window-B for Terminating
    - Height-B
    - Width-B
- Structure Simulation
  - Model Simulation
    - Equation of Motion -> State-Space Model
  - Excitation Simulation
    - Ambient Vibration
      - Gaussian Noise
    - Earthquake
      - Kanai-Tajimi
    - Impact
      - Spike-like Input
    - Strong Wind
      - Auto Regression
  - Measurement
    - Noise

### Estimator - *To Estimate the System State Based on Measurement*
#### Estimator - Analyses - *Try to Evaluate the Actual State of the System / Progress*
- Target: to estimate the system state
  - Recall = TP/(TP+FN)
    - The problem is no way to know FN
  - Precision = TP/(TP+FP)
    - The problem is no way to know FP
  - F-beta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)

#### Estimator - Solution
- Two Phases
  - Phase I - capture all the events
  - Phase II - only capture the events identified as event of interest by the classifier
- Assumptions
  - Perfect Classifier: all the events can be correctly classified
  - Imperfect Classifier: in practice, the classifier can never be 100% correct

### Planning - *To Provide The Ideal State As A Reference*

#### Planning - Analyses - *Key Concerns Of Planning*
- Key Point: 
  - High Recall - we don't want to miss out any important event
    - Recall: TP/(TP+FN) - the identified important events / all important events
  - High Precision - we want to reduce false alarm as much as possible
    - Precision: TP/(TP+FP) - the identified important events / all events identified as important events
  - How to strike a balance? F-beta score can be a good option. Then, the key is how to determine beta? 
    - F-beta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
      - large beta : more weight on recall; small beta : more weight on precision
        - F0.5-Measure (beta=0.5): More weight on precision, less weight on recall. - less power consumption
        - F1-Measure (beta=1.0): Balance the weight on precision and recall.
        - F2-Measure (beta=2.0): Less weight on precision, more weight on recall - more power consumption
      - The key is to determine an appropriate value for beta.

![F-beta](F-beta.png)

#### Planning - Solution - *To Maximize The Metric, But Don't Know How*
- Assume beta = 2 -> F2 score
- During the closed loop iteration, the planning block calculates the **ideal F score** - F-beta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
- F-2 = (5 * Precision * Recall) / (4 * Precision + Recall) = 5*R*P/(4P+R).
- The goal is to maximize the F-beta score, the higher the better. However:
  - We don't know where is the maximum.
  - We don't know the relationship between the F-beta score and the Recall and Precision.
- Assume only one peak for F-beta score curve
- We don't know the final optimal position, but we don't need to do that in one-shot. We only need to provide the ideal state for next move, and we can approach to the global optima by iteration.
- Starting from high Recall and low Precision, manipulate the triggering parameters, try to increase Precision while keep the Recall at a high level
- Greedy Search, Newton Method, Reinforced Learning... to find the maximum?

### Controller - *To Provide The Command To Guide The Next Move*

#### Controller - Analyses - *Key Concerns Of Control*
- For feedback control, the decision is made based on the gap between the ideal state and the current state, and the term for this gap is 'error'.
- Ideal State
  - The truth is we can never know the ideal state
    - For rule-based method, we can provide an estimated ideal state for each iteration.
    - For Data-driven method, like RL, together with the controller, the planning part can be incorporated with the controller.
- Current State
  - provided by the Estimator
#### Controller - Solutions
- if only the threshold is considered, PID can be used
- for multi-parameters to fine-tune, RL can be considered
